In [ ]:
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [ ]:
mdict = {"CAT " : CatBoostClassifier(verbose = False), "LGBM " : LGBMClassifier(), "RF " : RandomForestClassifier(),
        "LR " : LogisticRegression(solver = "liblinear"), "SGD " : SGDClassifier()}

In [ ]:
tree_grid1 = {"n_estimators" : [20, 40, 60, 80, 100],
            "learning_rate" : [0.01, 0.05, 0.1]}
tree_grid2 = {"n_estimators" : [20, 40, 60, 80, 100],
             "max_features" : ["log2", "sqrt"]}
linear_grid1 = {"penalty" : ["l2", "l1"], 
                "C" : [0.6, 0.7, 0.8, 0.9, 1.0]}
linear_grid2 = {"loss" : ["modified_huber", "squared_hinge", "perceptron"],
               "penalty" : ["l2", "l1"]}

pdict = {0 : tree_grid1, 1 : tree_grid1, 2 : tree_grid2, 3 : linear_grid1, 4 : linear_grid2}

In [ ]:
def make_confusion_matrix(DATA_PATH, DATA_PATH2, mdict):
    
    train, test = pd.read_csv(DATA_PATH), pd.read_csc(DATA_PATH2)
    test = train.loc[:, train.columns]
    
    model = mdict[m]
    
    X, y = train.drop(["target"], axis = 1), train["target"]
    X_test, y_test = test.drop(["target"], axis = 1), test["target"]
    
    grid = GridSearchCV(model, param_grid = pdict[idx], cv = 3, refit = True)
    grid.fit(X, y)
    _pred = grid.best_estimator_.predict(X_test)
                    
    mat = confusion_matrix(y_test, _pred)
    mat = pd.DataFrame(mat)
    mat.to_csv(DATA_PATH[:-4] + "-res.csv")